In [2]:
DF_PATH       = "../data/processed/2_cleaned_data.pkl"

ROLE_COLS      = ['DevType']
TECH_COLS      = ['LanguageHaveWorkedWith',
                  'DatabaseHaveWorkedWith',
                  'PlatformHaveWorkedWith',
                  'WebframeHaveWorkedWith',
                  'MiscTechHaveWorkedWith',
                  'ToolsTechHaveWorkedWith',
                  'NEWCollabToolsHaveWorkedWith']

MLFLOW_TRACKING_URI = '../models/mlruns'
MLFLOW_EXPERIMENT_NAME = "skills_jobs_stackoverflow"

LOG_PATH = "../models/temp/"
LOG_DATA_PKL    =  "data.pkl"
LOG_MODEL_PKL   =  "model.pkl"
LOG_METRICS_PKL =  "metrics.pkl"


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
from pathlib import Path

import mlflow
from  mlflow.tracking import MlflowClient


from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

    
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import StandardScaler


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score



In [4]:
df = pd.read_pickle(DF_PATH)
df

DevType                                           \
      Academic researcher Blockchain Cloud infrastructure engineer   
1                       0          0                             0   
2                       0          0                             0   
3                       0          0                             0   
4                       0          0                             0   
5                       0          0                             0   
...                   ...        ...                           ...   
89179                   0          0                             0   
89180                   0          0                             0   
89181                   0          0                             0   
89182                   0          0                             0   
89183                   0          0                             0   

                                                                              \
      Data or business analyst Data scientist or machine learning specialist   
1                            0                                             0   
2                            0                                             0   
3                            0                                             0   
4                            0                                             0   
5                            0                                             0   
...                        ...                                           ...   
89179                        0                                             0   
89180                        0                                             0   
89181                        0                                             0   
89182                        0                                             0   
89183                        0                                             0   

                                                                   \
      Database administrator DevOps specialist Developer Advocate   
1                          0                 0                  0   
2                          0                 0                  0   
3                          0                 0                  0   
4                          0                 0                  0   
5                          0                 0                  0   
...                      ...               ...                ...   
89179                      0                 0                  0   
89180                      0                 0                  0   
89181                      0                 0                  0   
89182                      0                 0                  0   
89183                      0                 0                  0   

                                                  ... skills_clusters  \
      Developer Experience Developer, QA or test  ... skills_group_21   
1                        0                     0  ...               0   
2                        0                     0  ...               0   
3                        0                     0  ...               0   
4                        0                     0  ...               0   
5                        0                     0  ...               0   
...                    ...                   ...  ...             ...   
89179                    0                     0  ...               0   
89180                    0                     0  ...               0   
89181                    0                     0  ...               0   
89182                    0                     0  ...               0   
89183                    0                     0  ...               0   

                                                                     \
      skills_group_22 skills_group_23 skills_group_3 skills_group_4   
1                   0               0              0              0   
2    

In [5]:
roles_df = df['DevType'].copy()
roles_df.sum(axis=0)

Academic researcher                               1279
Blockchain                                         299
Cloud infrastructure engineer                      994
Data or business analyst                           795
Data scientist or machine learning specialist     1528
Database administrator                             243
DevOps specialist                                 1319
Developer Advocate                                 200
Developer Experience                               308
Developer, QA or test                              571
Developer, back-end                              13402
Developer, desktop or enterprise applications     3765
Developer, embedded applications or devices       1807
Developer, front-end                              4974
Developer, full-stack                            24630
Developer, game or graphics                        838
Developer, mobile                                 2519
Engineer, data                                    1195
Hardware E

In [6]:
Sample_per_class = 1200
resampled_roles = []

for role in roles_df.columns :
    sub_df = roles_df.loc[roles_df[role]==1].copy()
    
    if len(sub_df) < Sample_per_class :
        sub_df = sub_df.sample(Sample_per_class , replace = True , random_state = 42)
    
    else :
        sub_df = sub_df.sample(Sample_per_class , random_state = 42)
    
    resampled_roles.append(sub_df)
        
roles_df = pd.concat(resampled_roles)

In [7]:
df = df.loc[roles_df.index].copy()

In [8]:
roles_df.sum(axis = 0)

Academic researcher                              1200
Blockchain                                       1200
Cloud infrastructure engineer                    1200
Data or business analyst                         1200
Data scientist or machine learning specialist    1200
Database administrator                           1200
DevOps specialist                                1200
Developer Advocate                               1200
Developer Experience                             1200
Developer, QA or test                            1200
Developer, back-end                              1200
Developer, desktop or enterprise applications    1200
Developer, embedded applications or devices      1200
Developer, front-end                             1200
Developer, full-stack                            1200
Developer, game or graphics                      1200
Developer, mobile                                1200
Engineer, data                                   1200
Hardware Engineer           

In [9]:

X_train, X_test, y_train, y_test = train_test_split( df.drop('DevType' , axis = 1), df['DevType'], test_size=0.33, random_state=42)

C:\Users\aly86\AppData\Local\Temp\ipykernel_20088\1303155953.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X_train, X_test, y_train, y_test = train_test_split( df.drop('DevType' , axis = 1), df['DevType'], test_size=0.33, random_state=42)


# Initialize MLflow

In [10]:
Path(MLFLOW_TRACKING_URI).mkdir(parents=True , exist_ok=True)
Path(LOG_PATH).mkdir(parents=True , exist_ok=True)

In [11]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()
mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
# Check if the experiment exists
exp = None
if exp is None:
    # Initialize a new experiment
    exp = client.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)


### Logistic regression ( Vanilla Model )

In [24]:
clf = make_pipeline(StandardScaler(),
                    MultiOutputClassifier(LogisticRegression()))

clf.fit(X_train.values, y_train.values)
predictions =  pd.DataFrame(clf.predict(X_train.values),columns=y_train.columns)

D:\End to End Data Science Projects\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\End to End Data Science Projects\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

In [25]:
predictions

,Academic researcher,Blockchain,Cloud infrastructure engineer,Data or business analyst,Data scientist or machine learning specialist,Database administrator,DevOps specialist,Developer Advocate,Developer Experience,"Developer, QA or test",...,"Developer, full-stack","Developer, game or graphics","Developer, mobile","Engineer, data",Hardware Engineer,Project manager,Research & Development role,Scientist,Security professional,System administrator
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19291,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19292,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19293,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19294,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
def calculate_quality(ground_truth, predictions, metric_function, sort_values=False):
    quality_scores = {}
    for col in predictions.columns:
        role_pred  = predictions[col].copy()
        role_truth = ground_truth[col].copy()
        quality_scores[col] = round(metric_function(role_truth, role_pred) * 100, 2)
        
    quality_scores = pd.Series(quality_scores.values(), index=quality_scores.keys())
    if sort_values:
        quality_scores = quality_scores.sort_values()
    
    return quality_scores

In [27]:
# Evaluate on training set
train_scores = {score.__name__: calculate_quality(y_train, predictions, score) 
                for score in [accuracy_score, precision_score, recall_score, f1_score]}
train_scores = pd.concat(train_scores,axis=1)

In [28]:
# Evaluate on test set
predictions =  pd.DataFrame(clf.predict(X_test.values), columns=y_test.columns)
test_scores = {score.__name__: calculate_quality(y_test, predictions, score) 
                for score in [accuracy_score, precision_score, recall_score, f1_score]}
test_scores = pd.concat(test_scores,axis=1)
mean_test_scores = test_scores.mean()

In [29]:
mean_test_scores



accuracy_score     96.058333
precision_score    50.666667
recall_score       19.440000
f1_score           26.272083
dtype: float64

In [30]:
# Data details
data_details = {"data_path": DF_PATH,
                "training_indices": X_train.index.tolist(),
                "test_indices":     X_test.index.tolist(), 
                "features_names":   X_train.columns.droplevel(0).tolist(),
                "targets_names":    y_train.columns.tolist()}

with open(os.path.join(LOG_PATH, LOG_DATA_PKL), "wb") as output_file:
    pickle.dump(data_details, output_file)

In [31]:
# Model
model = {"model_description": "Baseline model: Logistic Regression ",
         "model_details": str(clf),
         "model_object": clf} 

with open(os.path.join(LOG_PATH, LOG_MODEL_PKL), "wb") as output_file:
    pickle.dump(model, output_file)

In [32]:
# Performance details
classes_metrics = {"train_scores": train_scores, 
                   "test_scores":  test_scores}

with open(os.path.join(LOG_PATH, LOG_METRICS_PKL), "wb") as output_file:
    pickle.dump(classes_metrics, output_file)

In [33]:
# Start a new run and track 
with mlflow.start_run(experiment_id=exp.experiment_id, run_name=model["model_description"]):
    # Log pickles 
    mlflow.log_artifacts(LOG_PATH)
    
    # Track metrics 
    for metric, score in mean_test_scores.items():
        mlflow.log_metric(metric, score) 
   

In [34]:
exp.experiment_id

'372234719956380473'